# LSTM Tutorial Medium

Quelle: https://pub.aimind.so/generating-text-with-lstm-networks-using-tensorflow-a-comprehensive-guide-79cc2eeda20a

Code: https://github.com/sidharth72/Text-Generation-With-LSTM/blob/main/LSTM%20Text%20Generation/LSTM_Text_Generation.ipynb

In [2]:

import numpy as np
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Sequential
import tensorflow as tf
     

2024-11-06 17:27:21.090730: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-06 17:27:21.991613: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 17:27:21.991722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 17:27:22.155581: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 17:27:22.471907: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
with open('../../exaple_text.txt', 'r') as f:
    sentences = f.readlines()


print(sentences)

FileNotFoundError: [Errno 2] No such file or directory: '../exaple_text.txt'

In [ ]:

import re

def preprocess(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'\d+', '', sentence)
    sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
    sentence = sentence.strip()
    return sentence

sentences = [preprocess(sentence) for sentence in sentences]

print(sentences)

['wie geht es dir', 'mir gehts gut danke', 'was machst du heute', 'hast du gut geschlafen', 'wo wohnst du', 'ich wohne in berlin', 'wie spt ist es', 'es ist schon ziemlich spt', 'hast du heute zeit', 'was hast du am wochenende vor', 'kannst du mir bitte helfen', 'was hast du gestern gemacht', 'ich habe heute frei', 'hast du hunger', 'mchtest du etwas trinken', 'wo arbeiten sie', 'ich arbeite als lehrer', 'wann treffen wir uns', 'ich freue mich dich zu sehen', 'das wetter ist heute schn oder', 'es ist ziemlich kalt drauen', 'es regnet seit stunden', 'kannst du das fenster schlieen', 'mchtest du ins kino gehen', 'was fr musik hrst du gerne', 'hast du den film schon gesehen', 'ich habe heute viel zu tun', 'kommst du mit', 'hast du das schon gehrt', 'ich muss noch einkaufen gehen', 'was isst du zum mittagessen', 'ich habe keine lust zu kochen', 'hast du einen moment zeit', 'was hast du gesagt', 'kannst du das bitte wiederholen', 'ich habe es nicht verstanden', 'kannst du lauter sprechen', 

In [ ]:
def adjust_sentence_length(sentences, sequence_length):
    adjusted_sentences = []
    buffer = []

    for sentence in sentences:
        words = sentence.split()
        buffer.extend(words)

        while len(buffer) >= sequence_length:
            adjusted_sentences.append(' '.join(buffer[:sequence_length]))
            buffer = buffer[sequence_length:]

    if buffer:
        adjusted_sentences.append(' '.join(buffer))

    # Post-processing to ensure all sentences are around sequence_length
    adjusted_sentences_final = []
    for sentence in adjusted_sentences:
        words = sentence.split()
        if len(words) < sequence_length:
            if adjusted_sentences_final:
                last_sentence_words = adjusted_sentences_final[-1].split()
                if len(last_sentence_words) + len(words) <= sequence_length:
                    adjusted_sentences_final[-1] += ' ' + sentence
                    continue
        adjusted_sentences_final.append(sentence)

    return adjusted_sentences_final

sentences = adjust_sentence_length(sentences, 20)

print(sentences)

['wie geht es dir mir gehts gut danke was machst du heute hast du gut geschlafen wo wohnst du ich', 'wohne in berlin wie spt ist es es ist schon ziemlich spt hast du heute zeit was hast du am', 'wochenende vor kannst du mir bitte helfen was hast du gestern gemacht ich habe heute frei hast du hunger mchtest', 'du etwas trinken wo arbeiten sie ich arbeite als lehrer wann treffen wir uns ich freue mich dich zu sehen', 'das wetter ist heute schn oder es ist ziemlich kalt drauen es regnet seit stunden kannst du das fenster schlieen', 'mchtest du ins kino gehen was fr musik hrst du gerne hast du den film schon gesehen ich habe heute', 'viel zu tun kommst du mit hast du das schon gehrt ich muss noch einkaufen gehen was isst du zum', 'mittagessen ich habe keine lust zu kochen hast du einen moment zeit was hast du gesagt kannst du das bitte', 'wiederholen ich habe es nicht verstanden kannst du lauter sprechen wie lange dauert das noch was kostet das wo finde', 'ich die nchste ubahnstation gibt 

In [ ]:
def generate_vocab_map(d, sentences):
    index = 1
    for sentence in sentences:
        for word in sentence.split():
            if word not in d:
                d[word] = index
                index += 1

vocab_mapping = {}
generate_vocab_map(vocab_mapping, sentences)



In [ ]:

def convert_int_to_text(int_sequences, vocab_map):
    # Create a reverse mapping from index to word
    reverse_vocab_map = {index: word for word, index in vocab_map.items()}

    text_sequences = []
    for int_sequence in int_sequences:
        text_sequence = []
        for token in int_sequence:
            if token in reverse_vocab_map:
                text_sequence.append(reverse_vocab_map[token])

        text_sequences.append(' '.join(text_sequence))

    return text_sequences

In [ ]:
sequences = []

def generate_tokens(tokens_list, sentences, vocab_map):
    for sentence in sentences:
        sentence_tokens = []
        for word in sentence.split():
            if word in vocab_map:
                sentence_tokens.append(vocab_map[word])

        tokens_list.append(sentence_tokens)

generate_tokens(sequences, sentences, vocab_mapping)

print(sequences)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 11, 7, 14, 15, 16, 11, 17], [18, 19, 20, 1, 21, 22, 3, 3, 22, 23, 24, 21, 13, 11, 12, 25, 9, 13, 11, 26], [27, 28, 29, 11, 5, 30, 31, 9, 13, 11, 32, 33, 17, 34, 12, 35, 13, 11, 36, 37], [11, 38, 39, 15, 40, 41, 17, 42, 43, 44, 45, 46, 47, 48, 17, 49, 50, 51, 52, 53], [54, 55, 22, 12, 56, 57, 3, 22, 24, 58, 59, 3, 60, 61, 62, 29, 11, 54, 63, 64], [37, 11, 65, 66, 67, 9, 68, 69, 70, 11, 71, 13, 11, 72, 73, 23, 74, 17, 34, 12], [75, 52, 76, 77, 11, 78, 13, 11, 54, 23, 79, 17, 80, 81, 82, 67, 9, 83, 11, 84], [85, 17, 34, 86, 87, 52, 88, 13, 11, 89, 90, 25, 9, 13, 11, 91, 29, 11, 54, 30], [92, 17, 34, 3, 93, 94, 29, 11, 95, 96, 1, 97, 98, 54, 81, 9, 99, 54, 15, 100], [17, 101, 102, 103, 104, 3, 105, 106, 107, 108, 1, 109, 17, 84, 110, 111, 17, 78, 112, 113], [22, 54, 114, 1, 97, 115, 11, 81, 105, 17, 109, 116, 117, 17, 80, 118, 119, 120, 50, 121], [122, 11, 25, 13, 17, 34, 123, 124, 125, 15, 13, 11, 54, 126, 17, 100, 54, 127, 56, 13], [11, 23, 12

In [ ]:
tokens_list = [sequence for sequence in sequences]
tokens = [token for word_seq in tokens_list for token in word_seq]

tokens[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
input_sequence = []
targets = []
sequence_length = 20

for i in range(len(tokens) - sequence_length):
    input_sequence.append(tokens[i:i+sequence_length])
    targets.append(tokens[i + sequence_length])

print(input_sequence)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 11, 7, 14, 15, 16, 11, 17], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 11, 7, 14, 15, 16, 11, 17, 18], [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 11, 7, 14, 15, 16, 11, 17, 18, 19], [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 11, 7, 14, 15, 16, 11, 17, 18, 19, 20], [5, 6, 7, 8, 9, 10, 11, 12, 13, 11, 7, 14, 15, 16, 11, 17, 18, 19, 20, 1], [6, 7, 8, 9, 10, 11, 12, 13, 11, 7, 14, 15, 16, 11, 17, 18, 19, 20, 1, 21], [7, 8, 9, 10, 11, 12, 13, 11, 7, 14, 15, 16, 11, 17, 18, 19, 20, 1, 21, 22], [8, 9, 10, 11, 12, 13, 11, 7, 14, 15, 16, 11, 17, 18, 19, 20, 1, 21, 22, 3], [9, 10, 11, 12, 13, 11, 7, 14, 15, 16, 11, 17, 18, 19, 20, 1, 21, 22, 3, 3], [10, 11, 12, 13, 11, 7, 14, 15, 16, 11, 17, 18, 19, 20, 1, 21, 22, 3, 3, 22], [11, 12, 13, 11, 7, 14, 15, 16, 11, 17, 18, 19, 20, 1, 21, 22, 3, 3, 22, 23], [12, 13, 11, 7, 14, 15, 16, 11, 17, 18, 19, 20, 1, 21, 22, 3, 3, 22, 23, 24], [13, 11, 7, 14, 15, 16, 11, 17, 18, 19, 20, 1, 21, 22, 3, 3, 22, 23, 24, 21], [11, 7, 14,

In [ ]:

X = np.array(input_sequence)
y = np.array(targets)

In [ ]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(y)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(vocab_mapping) + 1, output_dim=128))
model.add(LSTM(128, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(128))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(units=len(vocab_mapping) + 1, activation='softmax'))

model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy'])

In [ ]:

model.fit(X, y, batch_size = 32, epochs = 100)

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 79ms/step - accuracy: 0.0047 - loss: 5.8599
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.0152 - loss: 5.3436
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.0378 - loss: 4.9442
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.0546 - loss: 4.6376
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.1243 - loss: 4.3096
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.1117 - loss: 4.1419
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.1434 - loss: 3.9168
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.2218 - loss: 3.6778
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.2345 - loss: 3.4427
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.2611 - loss: 3.2069
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.3125 - loss: 3.0322
Epoch 12/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/ste

In [ ]:
model.save('models/lstm_2.h5')

In [ ]:

def predict_next_word(input_text):
    input_text = input_text.lower()
    word_tokens = input_text.split()
    int_tokens = [vocab_mapping[token] for token in word_tokens]

    prediction = model.predict([int_tokens])
    prediction_idx = np.argmax(prediction)
    return convert_int_to_text([[prediction_idx]], vocab_mapping)[0]


def generate_text(input_text, n_words):
    word_sequence = input_text.split()
    context = word_sequence[:]
    for _ in range(n_words):
        prediction = predict_next_word(' '.join(context))
        word_sequence.append(prediction)
        context.append(prediction)
        if len(context) > 20:
            context.pop(0)

    return ' '.join(word_sequence)

In [ ]:
generate_text("everyone is living happy", 10)

KeyError: 'everyone'